In [1]:
import pandas as pd
import re

In [2]:
client_info_df = pd.read_csv(r'E:\wd_work\unti_cheat\D1_B_0201_0613_0615_B.csv')

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
my_contact_df = client_info_df[client_info_df.ACTION_CONTROL==u'my_contact']

In [7]:
my_contact_df1 = my_contact_df[['USER_ID','ID','ACTION_VALUE']].sort_values(by=['USER_ID','ID'])
my_contact_df1.ACTION_VALUE = my_contact_df1.ACTION_VALUE.apply(lambda x:re.sub(u"[a-z？]",'',str(x)))
my_contact_df1.ACTION_VALUE = my_contact_df1.ACTION_VALUE.apply(lambda x:re.sub(u"[a-z0-9]",'',str(x)))
my_contact_df1.ACTION_VALUE = my_contact_df1.ACTION_VALUE.apply(lambda x:re.sub(u"[？ ']",'',str(x)))

In [13]:
my_contact_df1['my_contact_len'] = my_contact_df1.ACTION_VALUE.str.len()
my_contact_df2 = my_contact_df1[my_contact_df1['my_contact_len']>=2].drop_duplicates(['USER_ID','ACTION_VALUE'])
my_contact_df3 = pd.merge(my_contact_df2,my_contact_df2[['USER_ID','ID']].groupby('USER_ID').count().reset_index().rename(columns={'ID':'ID_NUM'}),on='USER_ID',how='left')

In [15]:
my_contact_df4 = my_contact_df3[my_contact_df3.ID_NUM==1]

In [18]:
my_contact_df5 = my_contact_df3[my_contact_df3.ID_NUM!=1]

In [29]:
import difflib
def get_contact_cnt(df):
    my_contact_list = list(df.ACTION_VALUE)
    my_contact = [my_contact_list[-1],]
    for con in my_contact_list[:-1]:
        if con not in my_contact_list[-1] and difflib.SequenceMatcher(None,con,my_contact_list[-1]).ratio()<0.4:
            my_contact.append(con)
    return len(my_contact)
my_contact_df6 = my_contact_df5.groupby('USER_ID').apply(get_contact_cnt).reset_index().rename(columns={0:'my_contact_num'})

In [30]:
my_contact_df7 = my_contact_df4[['USER_ID','ID_NUM']].rename(columns={'ID_NUM':'my_contact_num'})

In [31]:
my_contact_df8 = pd.concat([my_contact_df6,my_contact_df7])

In [ ]:
#################联系人关系变更次数##########################

In [33]:
my_contact_relation_df = client_info_df[client_info_df.ACTION_CONTROL==u'my_contact_relation']

In [42]:
contactrelationship_changed_cnt= my_contact_relation_df[['USER_ID','ID','ACTION_VALUE']].groupby('USER_ID').count().reset_index()[['USER_ID','ID']].rename(columns={'ID':'contactrelationship_changed_cnt'})

In [43]:
###############联系人输入熟练情况############################

In [44]:
%%time
my_contact_df = client_info_df[client_info_df.ACTION_CONTROL=='my_contact']
my_contact_df = my_contact_df.sort_values(by=['USER_ID','ID'])
my_contact_df['my_contact_df_len'] = my_contact_df.ACTION_VALUE.str.len()
def cal_pro(df):
    df['my_contact_df_len1'] = df['my_contact_df_len'].shift(-1).fillna(0)
    df['dif'] = df['my_contact_df_len1']-df['my_contact_df_len']
    df1 = df[df.dif>0]
    if len(df1)>0:
        a = sum(df1.dif)/len(df1)
    else:
        a = -1
    return a
input_contact_proficiency_df = my_contact_df.groupby('USER_ID').apply(cal_pro)

Wall time: 52.6 s


In [45]:
###############是否有将手机输成联系人######################################

In [48]:
my_contact_df = client_info_df[client_info_df.ACTION_CONTROL=='my_contact'].sort_values(by=['USER_ID','ID'])[['USER_ID','ID','ACTION_VALUE']]

In [58]:
def get_mix_cnt(df):
    a = 0
    aaa = ','.join([str(aa) for aa in list(df.ACTION_VALUE)])
    for i in ['0','1','2','3','4','5','6','7','8','9']:
        if i in aaa:
            a = 1
    return a

In [62]:
is_phonecontact_mixed_df = my_contact_df.groupby('USER_ID').apply(get_mix_cnt).reset_index().rename(columns={0:'is_phonecontact_mixed'})

In [63]:
################is_hist_contactphone_black######################

In [64]:
my_contact_df = client_info_df[client_info_df.ACTION_CONTROL=='my_contact_phone'].sort_values(by=['USER_ID','ID'])[['USER_ID','ID','ACTION_VALUE']]

In [66]:
my_contact_df['ACTION_VALUE_LEN'] = my_contact_df.ACTION_VALUE.str.len()

In [75]:
my_contact_df1 = my_contact_df[my_contact_df['ACTION_VALUE_LEN']>=11]
is_hist_contactphone_black_df = my_contact_df1[my_contact_df1.duplicated('USER_ID',keep='last')]

In [ ]:
###################是否有复制粘贴##############################

In [77]:
%%time
my_contact_df = client_info_df[client_info_df.ACTION_CONTROL=='my_contact_phone']
my_contact_df = my_contact_df.sort_values(by=['USER_ID','ID']).drop_duplicates('USER_ID',keep='first')[['USER_ID','ACTION_VALUE']]
my_contact_df['my_contact_phone_len'] = my_contact_df.ACTION_VALUE.str.len()
my_contact_df['is_contact_phone_len_copypaste'] = 0
my_contact_df.ix[my_contact_df['my_contact_phone_len']>=5,'is_contact_phone_len_copypaste'] = 1
is_contact_phone_len_copypaste = my_contact_df[['USER_ID','is_contact_phone_len_copypaste']]

Wall time: 602 ms


In [ ]:
##################是否更换电话所属城市###########################

In [100]:
%%time
my_contact_df = client_info_df[client_info_df.ACTION_CONTROL=='my_contact_phone']
my_contact_df = my_contact_df.sort_values(by=['USER_ID','ID'])[['USER_ID','ACTION_VALUE']]
my_contact_df['ACTION_VALUE_LEN'] = my_contact_df.ACTION_VALUE.str.len()
my_contact_df1 = my_contact_df[my_contact_df['ACTION_VALUE_LEN']>=11]
my_contact_df1.ACTION_VALUE = my_contact_df1.ACTION_VALUE.apply(lambda x:re.sub(u"[^0-9]",'',str(x)))
my_contact_df1.ACTION_VALUE = my_contact_df1.ACTION_VALUE.apply(lambda x:x[:7])

Wall time: 743 ms


D:\Anaconda\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [101]:
phonecity_changed_cnt = my_contact_df1.drop_duplicates(['USER_ID','ACTION_VALUE']).groupby('USER_ID').count().reset_index()[['USER_ID','ACTION_VALUE']].rename(columns={'ACTION_VALUE':'phonecity_changed_cnt'})